# User Guide

## Introduction

The purpose of this guide is to explain the basic concepts of the *scarlet* package and how they are used. We also show how they can be extended and customized for more specialized science cases.
The [API Documentation](api_docs.rst) contains more detailed descriptions of the modules and classes used in *scarlet*, and a more rigorous overview of the mathematics and algorithms used by *scarlet* is described in [Moolekamp & Melchior 2018](https://arxiv.org/abs/1708.09066) and [Melchior et al. 2018](https://arxiv.org/abs/1802.10157).

### Basic Concepts and Structure

*scarlet* is designed to separate sources in astrophysical images by assuming that each scene can be thought of as a collection of multiple [Source](source.ipynb#scarlet.source.Source) objects.
Each [Source](source.ipynb#scarlet.source.Source) can be made up of multiple components, where each component has a single morphology (shape) and uniform spectrum (or SED) with a set of [Constraints](constraints.ipynb#scarlet.constraints.Constraint) they need to obey.
The [Source](source.ipynb#scarlet.source.Source) class is a base class, from which specialized classes can be derived to adjust to the sitation at hand. 
This customization can comprise the number of source components, their initialization, and the constraints they need to obey, or all of those.

The [Blend](blend.ipynb#scarlet.blend.Blend) class contains all of the information about the scene, i.e. the collection of Sources, as well as routines for fitting the joint model to data.
It implements the minimization algorithm described in [Moolekamp & Melchior 2018](https://arxiv.org/abs/1708.09066).
Below is a very brief summary of the method:

The deblending algorithm forms a model of the scene

$$\mathsf{M}= \sum_{k=1}^K \mathsf{A}_k^T \times \mathsf{S}_k = \mathsf{A}\mathsf{S}, $$

where $\mathsf{A}_k \in \mathbb{R}^B$ is the normalized SED and $\mathsf{S}_k \in \mathbb{R}^N$ is the morphology of a single component in the model with $B$ bands and $N$ pixels in each band.
It is important to note that this matrix factorization implies that SEDs and morphologies are independent, e.g. the SED of a component does not change over the region covered by its morphology.

The scene is fit by minimizing the likelihood of the model, namely minimizing

$$f(\mathsf{A},\mathsf{S}) = \frac{1}{2} || \mathsf{Y}-\mathsf{A}\mathsf{S} ||_2^2, $$

where $\mathsf{Y}$ is an image cube and $||.||_2$ is the element-wise $L_2$ (Frobenius) norm.

Each component $j$ can have $M_j$ different constraint functions $g_{ji}$, equivalent to minimizing

$$f(\mathsf{A}, \mathsf{S}) + \sum_{j=1}^K \sum_{i=1}^{M_j} g^A_{ji} \left(\mathsf{A}_{ji} \right) + g^S_{ji} \left(\mathsf{S}_{ji} \right)$$

Those constraints are applied to each source in the form of proximal operators, a handy mathematical approach for imposing non-smooth constraints that (if properly formulated) are guaranteed to converge; the curious reader will find more details in [Parikh & Boyd 2014](http://www.web.stanford.edu/~boyd/papers/pdf/prox_algs.pdf) and [Combettes & Pesquet 2011](https://link.springer.com/chapter/10.1007/978-1-4419-9569-8_10).
In short, proximal operators map an input vector to the nearest vector that satisfied the respective constraint.
Many constraints/penalty functions have analytic proximal operators. 

*scarlet* allows for two different kinds of constraints: proximal operators in the direct domain (that is directly on the SEDs $\mathsf{A}_k$ and morphologies $\mathsf{S}_k$) or in the transformed domain, i.e. after SEDs and morphologies are transformed by a linear operator $\mathsf{L}$ that is specified by the user. Any linear operator is allowed (it does not have to be invertible), examples are finite differences or basis transforms. The main reason for working in the transformed domain is that it can be much easier to express the constraint, for instance that a particular solution have only one Fourier coefficient.

Proximal operators for the direct and transformed domain behave differently. The former can be thought of as *strictly* obeyed at every iteration, while the latter ones will converge to a solution that obeys the constraint eventually. 
To see this it is useful to understand how $\mathsf{A}$ and $\mathsf{S}$ are updated:

$$ x^{\textrm{it}+1} \leftarrow \textrm{prox}_{\lambda f} \left( x^{\textrm{it}} - \frac{\lambda}{\rho} \mathsf{L}^T \left( \mathsf{L} x^{\textrm{it}}-z^{\textrm{it}} + u^{\textrm{it}} \right) \right)$$

$$z^{\textrm{it}+1} \leftarrow \textrm{prox}_{\rho g} \left( \mathsf{L} x^{\textrm{it}+1} + u^{\textrm{it}} \right)$$

$$u^{\textrm{it}+1} \leftarrow u^{\textrm{it}} + \mathsf{L} x^{\textrm{it}+1} - z^{\textrm{it}+1}$$

where $x^{\textrm{it}}$ denotes either $\mathsf{A}_k$ or $\mathsf{S}_k$ in the current iteration, and $\textrm{prox}_f$ performs (at least) a step in the direction of the negative gradient of the log-likelihood. $\textrm{prox}_g$ is the proximal operator in the transformed domain as it acts on $\mathsf{L} x$ instead of $x$.
If additional constraints in the direct domain are present, they will be subsumed in $\textrm{prox}_f$, thus making sure that they are satisfied in every iteration.

## Sources

The base [Source](source.ipynb#scarlet.source.Source) class requires an initial guess for the SED and morphology of each source (in fact, a list of seds and morphologies to support multi-component sources, e.g. bulge-disc models).
Most users might want to use derived classes [PointSource](source.ipynb#scarlet.source.PointSource) and [ExtendedSource](source.ipynb#scarlet.source.ExtendedSource), which initialize with a first guess of the morphology and SED from the image cube, and use symmetry, monotonicity, and positivity constraints.
Users with more specific needs can create [custom sources](#Custom-Sources).

### Object Detection

Before we start, we need to load an example image (here an image cube with 5 bands) *and* a detection catalog.
If such a catalog is not available, packages like [SEP](http://sep.readthedocs.io/) or [photutils](https://photutils.readthedocs.io/en/stable/) will happily generate one.
Instead, we will simply load the truth catalog of the simulation.
While not fundamentally needed, *scarlet* works best if the background noise levels are known as well, so we'll also set it to the truth value:

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
# use a better colormap and don't interpolate the pixels
matplotlib.rc('image', cmap='inferno')
matplotlib.rc('image', interpolation='none')

import numpy as np
import scarlet

# Load the sample images
data = np.load("../data/test_sim/data.npz")
images = data["images"]

# Load the thruth detection catalog
from astropy.table import Table as ApTable
catalog = ApTable.read("../data/test_sim/true_catalog.fits")
bg_rms = np.array([20]*len(images))

### Initialization

Fundamentally, a [Source](source.ipynb#scarlet.source.Source) needs to hold an SED and morphology.
A simple, one-pixel [Source](source.ipynb#scarlet.source.Source) with a SED taken from the center of the image can be set up like so:

In [ ]:
B, Ny, Nx = images.shape
center = (Ny//2, Nx//2)
# Get the SED at the location of the central pixel, format (1, B) for one component
sed = np.expand_dims(scarlet.source.get_pixel_sed(images, center), axis=0)
# Set the morphology such that only the central pixel has any intensity
morph = np.zeros((Ny, Nx))
morph[center] = 1
morph = np.expand_dims(morph, axis=0) # format (1, Ny, Nx) for one component
src = scarlet.Source(sed=sed, morph_image=morph)

Sources will typically have a `center` (position) argument, but it's not required as some source types are not (well) localized.

The location of the center can be re-adjusted to improve localization and to better satisfy constraints like symmetry and monotonicity, which are evaluated with respect to the center.
This is done by calculating the dipole of the residuals when shifting the model by `shift_center`, typically a fraction of a pixel.
If there is a good reason to believe that the initial positions are correct (such as in simulations), one can set `shift_center=0`.

In general, most sources only cover a small region of the full image, it is therefore more computationally efficient to initialize the source in the smallest possible frame (or bounding box).
When the entire blend is fit, the size of this frame is recalculated if `fix_frame` is `False` (the default value).

One can also prevent the fitting procedure to prevent changes to SED or morphology by setting `fix_sed` or `fix_morph` to `True` (default is `False`).

While using the [Source](source.ipynb#scarlet.source.Source) base class is perfectly valid, it is more common to initialize a source using an inherited class.
For example, [PointSource](source.ipynb#scarlet.source.PointSource) creates a new source given a `center` position using the SED of the pixel at that location in each band with only that single pixel turned on in the morphology, which is a decent starting configuration for stars.
A more generally useful class is [ExtendedSource](source.ipynb#scarlet.source.ExtendedSource), which initializes a source with a symmetric and monotonic model around the `center` location, using the mean SED over that footprint.

For example, to create a list of extendend sources for every source in the input catalog:

In [ ]:
sources = [scarlet.source.ExtendedSource((src['y'],src['x']), images, bg_rms) for src in catalog]

Notice that [ExtendedSource](source.ipynb#scarlet.source.ExtendedSource) has a few additional arguments compared to the [Source](source.ipynb#scarlet.source.Source) base class.
It requires `img` to determine the initial SED and morphology and the background level `bg_rms` to determine to size of the frame (or bounding box).

## Constraints

### Introduction
*scarlet* is implemented in a modular and flexible way that allows users to implement custom constraints tailored to the situations at hand.

When specifying a constraint the user can choose what quantity is affected, and how. The [Constraint](constraints.ipynb#scarlet.constraints.Constraint) base class holds these members:
```
prox_sed
prox_morph
prox_g_sed
L_sed
prox_g_morph
L_morph
source
```
The operators are called `prox_<sed/morph>` in the direct domain and `prox_g_<sed/morph>` in the transformed domain, for reasons that should be clear from the equations [above](#Basic-Concepts-and-Structure).
Any of the members above can be `None`, which implies that they are mute.
If `L_<sed/morph> is None` and `prox_g_<sed/morph>` is set, we assume the the transformation matrix $\mathsf{L}$ is the identity.

Multiple constraints can be combined with the `&` operator into a [ConstraintList](constraints.ipynb#scarlet.constraints.ConstraintList). This allows for arbitrary combinations of constraints.
For example, the [MinimalConstraint](constraints.ipynb#scarlet.constraints.MinimalConstraint), which requires the SED to have non-negative elements that sum to unity and the morphology to be non-negative, can be combined with an [L0Constraint](constraints.ipynb#scarlet.constraints.L0Constraint) using

In [ ]:
import scarlet.constraints as sc
constraint = sc.MinimalConstraint() & sc.L0Constraint(0.1)
constraint

We can look at the direct constraints for the sed:

In [ ]:
constraint.prox_sed

which shows (as expected) that the sed must be positive and sum to unity.

We also see that there are no constraints in the transformed domain and consequently also no linear operators:

In [ ]:
print(constraint.prox_g_sed)
print(constraint.prox_g_morph)
print(constraint.L_sed)
print(constraint.L_morph)

But we get a surprise when we look at direct operator for the morphology:

In [ ]:
constraint.prox_morph

`proxmin.operators.AlternatingProjections` is a helper class that is automatically constructed when there are multiple direct-domain constraints (we asked for a L0 penalty *and* positivity).
The order *is* important.
We adopt the same ordering as with linear operators, starting with the right-most one of the `scarlet.constraints.ConstraintList`, here the L0 penalty, then the positivity:

In [ ]:
constraint.prox_morph.operators

Because the direct-domain constraints might not commute, in some caes the proximal operators in an `AlternatingProjections` object might want to be repeated several times in a single iterations of the fit.
To allow for repeated projections in a single iteration use

In [ ]:
constraint = sc.ConstraintList(constraints=[sc.MinimalConstraint(), sc.L0Constraint(0.1)], repeat=3)

to repeat all of the alternating projections `3` times in each iteration of *scarlet*.

### Symmetry

Demanding that astrophysical sources are symmetric reduces the number of effective degrees of freedom of the model, and most galaxies are *largely* symmetric.
The idea has been used successfully in the SDSS deblender and also in our tests on substantially deeper HSC images.

Symmetry can be enforced in either the direct or transformed domain, where the former has the ability to specify how strictly symmetry is enforced (for example grand design spirals, irregular galaxies, and jets are not perfectly symmetric and it can be useful to use a softer penalty).

#### Direct Symmetry

Direct symmetry imposes the [prox_soft_symmetry](operators.ipynb#scarlet.operators.prox_soft_symmetry) projection operator:

$$ \textrm{prox}_\textrm{sym} = \frac{\sigma}{2} \left( S_k + S_k^\dagger \right) + \left(1-\sigma \right) S_k $$

where $S_k$ is the flattened morphology matrix for a single source and $S_k^\dagger$ is it's symmetric version.
So the parameter $\sigma \in [0,1]$ determines the minimum symmetry required, where $\sigma=0$ imposes no symmetry constraint at all while $\sigma=1$ enforces perfect symmetry.

We can combine a direct symmetry constraint with the minimal constraint using

In [ ]:
constraint = sc.MinimalConstraint() & sc.DirectSymmetryConstraint(sigma=0.5)

#### Symmetry in the Transformed Domain

In the transformed domain symmetry is implemented using a linear matrix $\mathsf{L}=$[getSymmetryOp](transformations.ipynb#scarlet.transformations.getSymmetryOp), which encodes the difference between each pixel and it's symmetric partner, and the proximal operator `proxmin.operators.prox_zero` forces the transformed variable to be zero.
As stated in the previous subsection, this constraint is not strictly met in any given iteration but will converge over time to a solution with a symmetric morphology.

We can combine this constraint with the minimal `constraint` using

In [ ]:
constraint = sc.MinimalConstraint() & sc.SymmetryConstraint()

Now we see that our constraint has a `prox_g_morph` but still no `L_morph`

In [ ]:
print(constraint.prox_g_morph)
print(constraint.L_morph)

This is because the linear operator can't be built until the source is initialized (because it needs to know the shape of the morphology).
If we initialize a source with this constraint

In [ ]:
src = scarlet.source.PointSource((catalog["y"][0], catalog["x"][0]),
                                 img=images, shape=(15, 15),
                                 constraints=constraint)
print(constraint.L_morph)

we see that the linear operator has in fact been created.

### Monotonicity

Another useful constraint from the SDSS-HSC deblender is the approximation that most astrophysical objects are monotonically decreasing from the peak.
In detail this assumption is violated e.g. in spiral galxies, especially tightly wound ones.
But if we think of spirals as a single source made up of multiple components, each monotonically decreasing from it's peak with a single SED, we can build a model that is well representative of even morphologically complex galaxies.
This point of view has the added benefit that regions that are not monotonically decreasing in a galaxy are likely different stellar populations with (potentially) different SED's and should be treated as separate components anyway.

Monotonicity in *scarlet* is implemented in the direct and transformed domain, so we will look at each use to see their differences.
The easier one to understand is in the transformed domain: the [MonotonicityConstraint](constraints.ipynb#scarlet.constraints.MonotonicityConstraint) class builds a [getRadialMonotonicOp](transformations.ipynb#scarlet.transformations.getRadialMonotonicOp) linear operator that takes the differences between the flux in the reference pixels and the flux in the current pixel.
The transformed morphology vector is then passed on to the `proxmin.operators.prox_plus` proximal operator to project it onto the subspace where all values are non-negative.
If `use_nearest` is `True`, only a single reference pixel is used: the nearest one in the direction to the peak.
Otherwise a weighted average of all pixels closer to the peak than the current pixel is used to allow for a smoother monotonic solution.

| ![](images/nearest_ref.png) | ![](images/weighted_ref.png) |
|:---------------------------:|:----------------------------:|
| Nearest Neighbor            | Weighted Reference           |


In the following example, for simplicity we use the [PointSource](source.ipynb#scarlet.source.PointSource) class to create a source with a simple initial SED and morphology, but add both a [SymmetryConstraint](constraints.ipynb#scarlet.constraints.SymmetryConstraint) and a [MonotonicityConstraint](constraints.ipynb#scarlet.constraints.MonotonicityConstraint).

In [ ]:
# Initialize a constraint that is used for all of the sources
constraint = (
    sc.SimpleConstraint() # sed sum to unity, all elements of SED and morph are non-negative
    & sc.MonotonicityConstraint(use_nearest=False) # prox_g monotonicity
    & sc.DirectSymmetryConstraint() # prox_f perfect symmetry
)
# Initialize the Sources
sources = [scarlet.source.PointSource(
    (src['y'],src['x']), # center coordinates in `images`
    images, # data cube (bands, Ny, Nx)
    (15,15), # initial shape of the bounding box
    constraints=constraint.copy(), # A copy of the constraint
) for src in catalog]

Now we create a blended scene with all of the sources, and add a helper method to display the model nicely.
We'll discuss the [Blend](blend.ipynb#scarlet.blend.Blend) class later, but for now we just use it to run a few iterations to see the monotonicity constraint in action.

In [ ]:
import scarlet.display

# Display the sources
def display_sources(sources, norm=None, subset=None, combine=False, show_sed=True):
    """Display the data and model for all sources in a blend
    
    This convenience function is used to display all (or a subset) of
    the sources and (optionally) their SED's.
    """
    if subset is None:
        # Show all sources in the blend
        subset = range(len(sources))
    for m in subset:
        # Load the model for the source
        src = sources[m]
        model = src.get_model(combine=combine)
        # Since a model can be generated for each component in a source,
        # or all components can be combined into a single model,
        # reshape the model if there is only a single component
        if len(model.shape)==4:
            components = model.shape[0]
        else:
            model = np.expand_dims(model, axis=0)

        # Select the image patch the overlaps with the source and convert it to an RGB image
        img_rgb = scarlet.display.img_to_rgb(images[src.bb], filter_indices=[3,2,1], norm=norm)

        # Build a model for each component in the model
        rgb = []
        for _model in model:
            # Convert the model to an RGB image
            _rgb = scarlet.display.img_to_rgb(_model, filter_indices=[3,2,1], norm=norm)
            rgb.append(_rgb)

        # Display the image and model
        figsize = [6,3]
        columns = 2
        # Calculate the number of columns needed and shape of the figure
        if show_sed:
            figsize[0] += 3
            columns += 1
        if not combine:
            figsize[0] += 3*(model.shape[0]-1)
            columns += model.shape[0]-1
        # Build the figure
        fig = plt.figure(figsize=figsize)
        ax = [fig.add_subplot(1,columns,n+1) for n in range(columns)]
        ax[0].imshow(img_rgb)
        ax[0].set_title("Data: Source {0}".format(m))
        for n, _rgb in enumerate(rgb):
            ax[n+1].imshow(_rgb)
            if combine:
                ax[n+1].set_title("Initial Model")
            else:
                ax[n+1].set_title("Component {0}".format(n))
        if show_sed:
            for sed in src.sed:
                ax[-1].plot(sed)
            ax[-1].set_title("SED")
            ax[-1].set_xlabel("Band")
            ax[-1].set_ylabel("Intensity")
        # Mark the current source in the image
        y,x = src.center
        ax[0].plot(x-src.bb[2].start, y-src.bb[1].start, 'wx', mew=2)
        plt.tight_layout()
        plt.show()
        
blend = scarlet.Blend(sources, images, bg_rms=bg_rms)
blend.fit(50)

# Set the arcsinh color scaling object
asinh = scarlet.display.Asinh(img=images, Q=50)

# For simplicity only show the first 3 peaks
display_sources(sources, subset=[0,1,2], norm=asinh)

While sources 1 and 2 look ok, we see that source 0 is clearly not monotonically decreasing from the center; if you look closely, neither is source 1.

Now we'll use the direct monotonicity, which forces all of the pixels to be monotonically decreasing by starting at the center pixel and working radially outward to enforce monotonicity. This also comes in a weighted and nearest neighbor version, so for consistency we use the nearest neighbor monototonicity again:

In [ ]:
# Initialize a constraint that is used for all of the sources
constraint = (
    sc.SimpleConstraint() # sed sum to unity, all elements of SED and morph are non-negative
    & sc.DirectMonotonicityConstraint(use_nearest=False) # prox_f monotonicity
    & sc.DirectSymmetryConstraint() # prox_f perfect symmetry
)

# Initialize the Sources
sources = [scarlet.source.PointSource(
    (src['y'],src['x']), # center coordinates in `images`
    images, # data cube (bands, Ny, Nx)
    (15,15), # initial shape of the bounding box
    constraints=constraint.copy()
) for src in catalog]

# Create the blend and display the sources
blend = scarlet.Blend(sources, images, bg_rms=bg_rms)
blend.fit(50)
display_sources(sources, subset=[0,1,2], norm=asinh)

So we see significant improvement using the direct monotonicity, even though (for reasons beyond the scope of this document) it is not an exact proximal operator.
We have an `exact` version of the direct monotonicity, but it is too slow to be used in practice.

### Custom Sources

Many users may require specialized features that are not implemented in the existing [Source](source.ipynb#scarlet.source.Source) classes avilable so far in *scarlet*, so the framework is built to support substantial customization of the initialization and the constraint types.
For example, the following code creates a simple bulge-disk model:

In [ ]:
class BulgeDisk(scarlet.Source):
    """A galactic source with two components
    """
    def __init__(self, center, img, config=None):
        self.center = center
        if config is None:
            config = scarlet.Config()
        # Use the smallest cache size for the initial bounding box
        shape = (config.source_sizes[0],) * 2
        # Use the same constraints as the default `ExtendedSource`
        constraints = (sc.SimpleConstraint() &
                       sc.DirectMonotonicityConstraint(use_nearest=False) &
                       sc.SymmetryConstraint())
        # Initialize the SEDs and morphologies for both components
        sed, morph = self.make_initial(img, shape)
        # NOTE: Don't forget to initialize the `Source`, as there is a lot
        # of internal initialization
        super(BulgeDisk, self).__init__(sed, morph, center=center, constraints=constraints)

    def make_initial(self, img, shape):
        B, Ny, Nx = img.shape
        # A small but non-zero number
        tiny = 1e-10
        # center_int is a property of a `Source` that returns the
        # integer position of the coordinates
        _y, _x = self.center_int

        # Get the color of the center pixel
        disk_sed = scarlet.source.get_pixel_sed(img, self.center_int)
        # Turn on all of the pixels in the box for the disk
        disk_morph = np.zeros(shape)
        cy = shape[0] >> 1
        cx = shape[1] >> 1
        dy = shape[0] >> 2
        dx = shape[1] >> 2
        disk_morph[cy-dy:cy+dy+1, cx-dx:cx+dx+1] = max(img[:,_y,_x].sum(axis=0), tiny)
        disk_morph = disk_morph.reshape(shape[0]*shape[1])

        # Make the bulge SED redder (and normalize)
        bulge_sed = disk_sed + np.linspace(-0.1, 0.1, B)
        bulge_sed /= np.sum(bulge_sed)
        # Turn on a single pixel for the bulge
        bulge_morph = np.zeros(shape[0] * shape[1])
        center_pix = bulge_morph.size // 2
        bulge_morph[center_pix] = max(img[:,_y,_x].sum(axis=0), tiny)

        # Combine the two components into an initial `sed` and `morph`
        sed = np.array([bulge_sed, disk_sed])
        morph = np.array([bulge_morph, disk_morph]).reshape(2, shape[0], shape[1])
        return sed, morph

In this simple example a new bulge-disk model is initialized with only the central coordinates of the source and the image of the scene.
The source is initialized with the smallest available source size (see [Configuration](#Configuration)) and uses the same constraints as an [ExtendedSource](source.ipynb#scarlet.source.ExtendedSource), namely symmetry, monotonicity, and an SED that sums to unity (see [Constraints](#Constraints)).
The initial SED and morphology are defined in the `make_initial` method, which initializes the morphology with a disk component that is half the size of the initial bounding box and a bulge component, which is a single pixel slightly redder than the disk.
Finally the parent [Source](source.ipynb#scarlet.source.Source) class is initialized.

We can now create a list of sources with two components, initialized using the `BulgeDisk` class above.

In [ ]:
bd_sources = [BulgeDisk((src["y"], src["x"]), images) for src in catalog]
display_sources(bd_sources, subset=[0,1,2], norm=asinh)

If we run *scarlet* for a few iterations we see that the two components have begun to converge:

In [ ]:
bd_sources = [BulgeDisk((src["y"], src["x"]), images) for src in catalog]
blend = scarlet.Blend(bd_sources, images, bg_rms=bg_rms)
blend.fit(200)
display_sources(blend.sources, subset=range(3), norm=asinh)

## Blended Scenes

The [Blend](blend.ipynb#scarlet.blend.Blend) class contains the entire blended scence as well as the functions to fit to data.

### Configuration

A number of global configuration options, used by both [Blend](blend.ipynb#scarlet.blend.Blend) and [Source](source.ipynb#scarlet.source.Source) objects, are accessed through the [Config](config.ipynb#scarlet.config.Config) class.
See [Configuration](config.ipynb#Configuration-(scarlet.config)) for a detailed description of different configuration options.

Most of the properties can be modified by the user on the fly, but changing the `Config.source_sizes` propery should be accomplished by using the [Config.set_source_sizes](config.ipynb#scarlet.config.Config.set_source_sizes) method, which ensures that all of the `source_sizes` are valid (see [Config](config.ipynb#scarlet.config.Config) for more).

### Initialization

Initializing a new blended scene requires a list of `sources` ([Source](#scarlet.source.Source) objects), an image (`img`) datacube with dimensions (`bands`, `height`, `width`).
If a weightmap exists, the user can also pass a `weights` datacube with the same dimensions as `img`.
It is also useful to pass a value for the background RMS (`bg_rms`), which is used to adjust the box size needed for each source (see the discussion in [Adjusting Sources](#Adjusting-Sources)).
Finally an optional [Config](config.ipynb#scarlet.config.Config) class can be specified, with custom configuration options.

For most users, a good place to start is by defining each source as an [ExtendedSource](source.ipynb#scarlet.source.ExtendedSource) and initializing a blend with

In [ ]:
sources = [scarlet.source.ExtendedSource((src['y'],src['x']), images, bg_rms) for src in catalog]
blend = scarlet.Blend(sources, images, bg_rms=bg_rms)

This creates a scene with a collection of sources, each one with a single SED and a morphology that is both monotonic and symmetric (see [Sources](#Sources)), and will use `bg_rms` to minimize the box sizes of each source.

To customize the configuration of the [Blend](blend.ipynb#scarlet.blend.Blend), for example using a lower flux threshold for resizing source bounding boxes, a [Blend](blend.ipynb#scarlet.blend.Blend) can be initialized with a new [Config](config.ipynb#scarlet.config.Config).

In [ ]:
config = scarlet.Config(edge_flux_thresh=0.5)

which will resize the box if the model has flux along any edge that us greater than `Config.edge_flux_thresh*bg_rms`.

### Fitting a Model

The [Blend](blend.ipynb#scarlet.blend.Blend) class implement the minimization algorithm described in [Moolekamp & Melchior 2018](https://arxiv.org/abs/1708.09066). See details [above](#Basic-Concepts-and-Structure).

The [Blend.fit](blend.ipynb#scarlet.blend.Blend.fit) method will fit the current model and requires two parameters: the maximum number of `steps` (or iterations) used to fit the data and the relative error for convergence (`e_rel`).
It stops if one of the following conditions is met:

1. The total number of iterations is equal to `steps`

1. The model converges (as defined in  [Moolekamp & Melchior 2018](https://arxiv.org/abs/1708.09066)).


### Restarting a Fit

There may be instances where it is desirable to restart a fit.
For example, after a certain number of iterations inspect the result even prior to convergence, or you may have a custom constraint that you want to apply every Nth iteration.
In that case you can call `Blend.fit(N1)` and continue with another call to `Blend.fit(N2)`:

In [ ]:
sources = [scarlet.source.ExtendedSource((src['y'],src['x']), images, bg_rms) for src in catalog]
blend = scarlet.Blend(sources, images, bg_rms=bg_rms)
blend.fit(20)
print(blend.it)
blend.fit(20)
print(blend.it)
blend.fit(20)
print(blend.it)

where we see that the last fit the blend converged before reaching 60 iterations.

### Adjusting Sources

In most blends the number of non-zero pixels is much smaller than the total number of pixels in the image.
To save processing time we recommend initializing sources with a small number of pixels and allowing the [Blend.fit](blend.ipynb#scarlet.blend.Blend.fit) method to expand the size of the box.
This happens automatically for every source with `fix_frame=False` when that source's morphology has any flux above `config.edge_flux_thresh*blend.bg_rms` along the edge of its current bounding box.

With the default [symmetry](#Symmetry) and [monotonicity](#Monotonicity) constraints the model is dependent on the accuracy of the center position for each source.
The [Blend.recenter_sources](blend.ipynb#scarlet.blend.Blend.recenter_sources) method recenters all sources with `shift_center > 0` simultaneously by calculateing the shifts needed to align the models with the data to null any residual dipoles.

Both resizing sources and recentering sources are expensive operations (as resizing requires rebuilding all of the constraint operators and recentering requires building a shifted model) and are thus only performed every `Config.refine_skip` iterations.

### Accessing Individual Sources and Components

Internally the algorithm operates on the space of components, not sources (where a single [Source](#scarlet.source.Source) might have multiple components), so a [Blend](blend.ipynb#scarlet.blend.Blend) contains a number of indexing properties to convert from component space to source space.

For example, a `blend` created using the `BulgeDisk` class has two components for each source, for a total of 2$\times$sources components.

In [ ]:
bd_sources = [BulgeDisk((src["y"], src["x"]), images) for src in catalog]
blend = scarlet.Blend(bd_sources, images, bg_rms=bg_rms)
print("Number of sources:", blend.M)
print("Number of components:", blend.K)

If we iterate through the sources we can find the index of the component in `blend`:

In [ ]:
for m, src in enumerate(blend.sources):
    for l in range(src.K):
        k = blend.component_of(m,l)
        print("Component {0} has index {1}".format((m,l),k))

Similarly, we can use the index of the model component $k$ to find the index of the source, and the index of the component inside that source:

In [ ]:
for k in range(blend.K):
    m,l = blend.source_of(k)
    print("Component {0} is component {1} in source {2}".format(k, l, m))

[Blend](blend.ipynb#scarlet.blend.Blend) also has a `__len__` method, so taking the length of `blend` gives the number of sources (not components):

In [ ]:
len(blend)

## Loading and Displaying a Model

### Display functions

The [display](display.ipynb) module contains a number of convenience methods to convert an image cube into an RGB image array.

There are two stock classes used to scale the pixels in the image, [Linear](display.ipynb#scarlet.display.Linear) and [Asinh](display.ipynb#scarlet.display.Asinh), both of which inherit from the `matplotlib.colors.Normalize` class.
This inheritance allows them to be used as normalizations in `matplotlib.pyplot.imshow`, including an `inverse` method that makes it possible to add a colorbar.
For example:

In [ ]:
norm = scarlet.display.Linear(img=images)
plt.imshow(images[2], norm=norm)
plt.colorbar()
plt.title("Linear Scaling")
plt.show()

norm = scarlet.display.Asinh(img=images)
plt.imshow(images[2], norm=norm)
plt.colorbar()
plt.title("Asinh Scaling")
plt.show()

The [Linear](display.ipynb#scarlet.display.Linear) class is fairly straightforward, allowing the user to (optionally) pass `vmin` and `vmax` parameters to set the range of the data:

In [ ]:
norm = scarlet.display.Linear(vmin=0, vmax=100)
plt.imshow(images[2], norm=norm)
plt.colorbar()
plt.title("Linear Scaling")
plt.show()

The [Asinh](display.ipynb#scarlet.display.Asinh) scaling is popular because it is linear for small values and logrithmic for larger fluxes, allowing it to display a wide range of intensities clearly.
The actual formula used to scale each pixel is

$$f(x) = \frac{1}{Q} \sinh^{-1} \left( Q \frac{x-v_\textrm{min}}{v_\textrm{max}-v_\textrm{min}} \right)$$

where `Q` is a parameter that defines the strech of the scaling.

In [ ]:
norm = scarlet.display.Asinh(img=images, vmin=0, Q=10)
plt.imshow(images[2], norm=norm)
plt.colorbar()
plt.title("Asinh Scaling")
plt.show()

norm = scarlet.display.Asinh(img=images, vmin=0, Q=100)
plt.imshow(images[2], norm=norm)
plt.colorbar()
plt.title("Asinh Scaling")
plt.show()

An image cube can be converted from a (bands, height, width) array into an RGB image array using the [img_to_rgb](display.ipynb#scarlet.display.img_to_rgb) function.
This allows the user to specify a normalization (`norm`), `fill_value` (value to use for any masked pixels), and list of indices to map to R, G, B respectively (`filter_indices`).
For example the default is to map the first three bands in reverse order to RGB:

In [ ]:
img_rgb = scarlet.display.img_to_rgb(images, norm=asinh)
plt.imshow(img_rgb)

In this case the mapping is $r \rightarrow R$, $g \rightarrow G$, $u \rightarrow B$.
A more natural mapping (and the one used in most of this document) is $i \rightarrow R$, $r \rightarrow G$, $g \rightarrow B$:

In [ ]:
img_rgb = scarlet.display.img_to_rgb(images, filter_indices=[3,2,1], norm=asinh)
plt.imshow(img_rgb)

### Extracting Models from a Blend

You can load the models that make up a scene at any time, even if the blend has been initialized but not fit for a single iteration. For example:

In [ ]:
# Initialize the blend but don't fit the model
bd_sources = [BulgeDisk((src["y"], src["x"]), images) for src in catalog]
blend = scarlet.Blend(bd_sources, images, bg_rms=bg_rms)
model = blend.get_model()
img_rgb = scarlet.display.img_to_rgb(model, filter_indices=[3,2,1], norm=asinh)
plt.imshow(img_rgb)

It is also possible to load the model for each source individually:

In [ ]:
models = blend.get_model(
    combine=False, # Don't combine the model for each source together
    flat=False # Don't flatten the model, keeping each component separate
)
img_rgb = scarlet.display.img_to_rgb(models[0], filter_indices=[3,2,1], norm=asinh)
plt.imshow(img_rgb)
plt.show()

or even by component:

In [ ]:
# Load a model 
model = blend.get_model(
    m=0, # index of the first source
    combine=False, # Don't combine the model for each source together
    combine_source_components=False # Don't combine the components into a single model for each source
)
# Display the bulge
img_rgb = scarlet.display.img_to_rgb(model[0], filter_indices=[3,2,1], norm=asinh)
plt.imshow(img_rgb)
plt.show()
# Display the disk
img_rgb = scarlet.display.img_to_rgb(model[1], filter_indices=[3,2,1], norm=asinh)
plt.imshow(img_rgb)
plt.show()

You might also want to extract just the morphology (without convolving with the SED). To prevent unexpected results (i.e. the model having a different shape depending on the given parameters) the resulting model will still have the shape (`components`, `bands`, `height`, `width`), however there are only `components` different results.

In [ ]:
# Run a few iterations, jsut to make the model more interesting
blend.fit(100)
model = blend.get_model(
    m=0, # index of the first source
    combine=False, # Don't combine the model for each source together
    combine_source_components=False, # Don't combine the components into a single model for each source
    use_sed=False # Don't convolve with the SED
)
plt.imshow(model[0][1])
plt.show()
plt.imshow(model[0][3])
plt.show()
plt.imshow(model[1][1])
plt.show()
plt.imshow(model[1][2])
plt.show()

### Extracting Models from a Source

It is also possible to access the morphology of a given source directly, which is always centered:

In [ ]:
src = blend.sources[0]
plt.imshow(src.image[0])
plt.show()

plt.imshow(src.image[1])
plt.show()

where `src.image` is the same as `src.morph.reshape(src.K, src.Ny, src.Nx)`.
Notice how in this space only the part of the model inside the bounding box is given, since this is the space where the morphology lives.

We can also look at the SED for each source:

In [ ]:
for m, src in enumerate(blend.sources):
    # Only display the SED for the bulge components
    plt.plot(src.sed[0], label=m)
plt.legend()

It is also possible to extract the model for a given source:

In [ ]:
model = src.get_model()
_model = scarlet.display.img_to_rgb(model, filter_indices=[3,2,1], norm=asinh)
plt.imshow(_model)

or by component:

In [ ]:
model = src.get_model(combine=False)
_model = scarlet.display.img_to_rgb(model[0], filter_indices=[3,2,1], norm=asinh)
plt.imshow(_model)

or without recentering  or PSF convolution:

In [ ]:
Gamma = src._gammaOp([0,0], src.shape)
model = src.get_model(Gamma=Gamma)
_model = scarlet.display.img_to_rgb(model, filter_indices=[3,2,1], norm=asinh)
plt.imshow(_model)

This works because the `Source._gammaOp` is an operator that handles psf convolution and translation. This part of the code is likely to change in the near future, so we won't give a detailed description of how this is implementd in the code. But basically, `Source._gammaOp([dy,dx])` performs a psf convolution (if a psf was specified) and a linear shift by $dx$ and $dy$, the fraction of a pixel to move in the $x$ and $y$ directions respectively.

## Other Useful Source Properties

Below are a list of properties that can be accessed for a source. For more information about them, see [Source](source.ipynb#scarlet.source.Source).

In [ ]:
src = blend.sources[1]
print("Source shape:", src.shape)
print("Source dimensions:", (src.Ny, src.Nx))
print("Center:", src.center)
print("integer center:", src.center_int)
print("Slice of the original image to fit the source:", src.get_slice_for(images.shape))

Display the image centered on the source, where `Source.bb` is the bounding box to extract the source:

In [ ]:
_img = images[src.bb]
img_rgb = scarlet.display.img_to_rgb(_img, filter_indices=[3,2,1], norm=asinh)
plt.imshow(img_rgb)

If a source is on the edge then it might be necessary to extract only part of the full source using [Source.get_slice_for](source.ipynb#scarlet.source.Source.get_slice_for), where `source.image[source.get_slice_for(images.shape)]` is the same shape and aligned with `images[source.bb]`:

In [ ]:
fig = plt.figure(figsize=(6,3))
ax = [fig.add_subplot(1,2,1+n) for n in range(2)]
ax[0].imshow(img_rgb)
_img = scarlet.display.img_to_rgb(src.get_model()[src.get_slice_for(images.shape)], filter_indices=[3,2,1], norm=asinh)
ax[1].imshow(_img)

This concludes the overview. The user is referred to the [API Documentation](api_docs.rst) for more details about the objects used in *scarlet*.